In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from datetime import datetime as dt, timedelta
import numpy as np
import cytrader as bt
from river import metrics
from btCcxt import execute

from setEnv import set_binance_env
from cystratbase import CyStrategy
from cyutils.cyzers import ll_single_kelly
from models import logisticReg
from utils.flow import Kalman

In [ ]:
class StreamML(CyStrategy):
    def __init__(self):
        self.mean = dict()
        self.std = dict()
        self.kelly = dict()
        self.returns = dict()
        self.kalman = dict()
        self.ama = dict()
        self.boll = dict()
        self.x = dict()
        self.y = dict()
        self.y_pred = dict()
        self.model = dict()
        self.metric = dict()
        for i, d in enumerate(self.datas):
            tkr = self.datas[i]._name
            self.returns[tkr] = bt.ind.PercentChange(self.datas[i].close, period=1)
            self.mean[tkr] = bt.indicators.MovingAverageSimple(self.returns[tkr], period=self.p.kel_window)
            self.std[tkr] = bt.indicators.StdDev(self.returns[tkr], period=self.p.kel_window)
            
            self.kalman[tkr] = Kalman(tkr, interval=self.p.interval)
            self.ama[tkr] = bt.ind.KAMA(self.datas[i].close, fast=self.p.ama_fast, slow=self.p.ama_slow)
            self.boll[tkr] = bt.indicators.BollingerBands(period=self.p.bol_window, devfactor=2)
            self.model[tkr] = self.p.model().model
            self.metric[tkr] = metrics.Rolling(self.p.metric(), self.p.metric_window)
            self.x[tkr] = []
            self.y[tkr] = []
            self.y_pred[tkr] = []

    params = (
        ("live", True),
        ("futures", False),
        ("log", False),
        ("verbose", False),
        ("debug", False),
        ("log_file", f'../results/stream_{dt.now().strftime("%Y_%m_%d-%H_%M")}.csv'),
        ("interval", "1m"),
        ("model", logisticReg),
        ("metric", metrics.ROCAUC),
        ("metric_window", 12),
        ('n_positions', 1),
        ('min_positions', 0),
        ('kel_window', 30),
        ('kel_bounds', [0., 1.]),
        ("bol_window", 20),
        ("ama_fast", 6),
        ("ama_slow", 12),
        )

    def next(self):
        log = self.p.log
        if self.p.live:
            positions = [d for d, pos in self.getpositions().items() if pos]
        else:
            positions = [d._name for d, pos in self.getpositions().items() if pos]

        up, down = {}, {}
        for data in self.datas:
            t = data.datetime.datetime(0)
            tkr = data._name
            o = data.open[0]
            h = data.high[0]
            l = data.low[0]
            c = data.close[0]
            v = data.volume[0]
            returns = self.returns[tkr]
            boll = self.boll[tkr]
            ama = self.ama[tkr]
            model = self.model[tkr]

            # print("{} - {} | O: {} H: {} L: {} C: {} V:{}".format(t, tkr, o, h, l, c, v))

            kal = self.kalman[tkr]
            kal._update(t, returns)

            x = {
                "t": t,
                "o": o,
                "h": h,
                "l": l,
                "c": c,
                "v": v,
                "ret": returns[0],
                "kal": kal.state_mean.iloc[-1],
                "ama": ama[0],
                "bbh": boll.top[0],
                "bbm": boll.mid[0],
                "bbl": boll.bot[0],
            }
            self.x[tkr].append(x)

            if self.p.verbose:
                print(self.x[tkr])

            self.y_pred[tkr].append(model.predict_proba_one(self.x[tkr][-1]))

            if returns > 0:
                self.y[tkr].append(1)
            elif returns < 0:
                self.y[tkr].append(-1)
            else:
                self.y[tkr].append(0)
            
            if len(self.x[tkr]) > 1:
                model.learn_one(self.x[tkr][-2], self.y[tkr][-1])
                metric = self.metric[tkr]
                metric.update(self.y[tkr][-1], self.y_pred[tkr][-2])
                if log:
                    self.log(f'{self.p.metric.__name__}: {metric}')
                    print((f'{self.p.metric.__name__}: {metric}'))
                # evaluate.progressive_val_score(X_y, model, metric, print_every=20_000)

        for i, d in enumerate(self.datas):
            tkr = self.datas[i]._name
            y_pred = self.y_pred[tkr]
            if len(y_pred) > 1:
                pos = y_pred[-1][True]
                neg = y_pred[-1][False]
                proba = neg*-1 if neg > pos else pos if pos > neg else 0
                if proba > 0:
                    up[data._name] = proba
                elif proba < 0:
                    down[data._name] = proba
                if log:
                    self.log(f'PROB: {proba}')

                # try:
                self.kelly[tkr] = abs(proba)*ll_single_kelly(self.mean[tkr][0], self.std[tkr][0], bounds=np.array(self.p.kel_bounds))
                # except:
                #     self.kelly[tkr] = 0
                if log:
                    self.log(f'KELLY: {self.kelly[tkr]}')

        shorts = sorted(down, key=down.get)[:self.p.n_positions]
        longs = sorted(up, key=up.get, reverse=True)[:self.p.n_positions]
        n_shorts, n_longs = len(shorts), len(longs)

        if n_shorts < self.p.min_positions or n_longs < self.p.min_positions:
            longs, shorts = [], []
        for ticker in positions:
            if ticker not in longs + shorts:
                self.order_target_percent(data=ticker, target=0)
                if log:
                    self.log(f'{ticker} | CLOSING ORDER CREATED')
                    
        for tkr in shorts:
            short_target = -1 * self.kelly[tkr]
            if log:
                self.log(
                    f'{tkr} short target: {short_target}')
            self.order_target_percent(data=tkr, target=short_target)
            if log:
                self.log(f'{tkr} | SHORT ORDER CREATED')

        for tkr in longs:
            long_target = self.kelly[tkr]
            if log:
                self.log(
                    f'{tkr} long target: {long_target}')
            self.order_target_percent(data=tkr, target=long_target)
            if log:
                self.log(f'{tkr} | LONG ORDER CREATED')

    def order_target_value(self, data=None, target=0.0, price=None, **kwargs):
        '''
        See bt.Strategy.order_target_value()
        This function has been modified to order fractional position sizes for 
        trading cryptocurrency or forex, as opposed to fixed-integer-sized contracts.
        '''
        if isinstance(data, str):
            data = self.getdatabyname(data)
        elif data is None:
            data = self.data

        possize = self.getposition(data, self.broker).size
        print("possize:", possize)
        if not target and possize:  # closing a position
            return self.close(data=data, size=possize, price=price, **kwargs)

        else:
            value = self.broker.getvalue(datas=[data])
            comminfo = self.broker.getcommissioninfo(data)

            # Make sure a price is there
            price = price if price is not None else data.close[0]
            if self.p.debug:
                print(f"TARRGET: {target}, VALUE: {value}")
                self.log(f"TARRGET: {target}, VALUE: {value}")
            if target > value:
                trgt = target - value
                size = comminfo.get_fracsize(price, trgt)
                if self.p.debug:
                    self.log(f"SIZE: {size}")
                return self.buy(data=data, size=size, price=price, **kwargs)

            elif target < value:
                trgt = value - target
                size = comminfo.get_fracsize(price, trgt)
                if self.p.debug:
                    self.log(f"SIZE: {size}")
                if not self.broker.futures:
                    if possize > size:
                        return self.sell(data=data, size=size, price=price, **kwargs)
                    else:
                        return
                return self.sell(data=data, size=size, price=price, **kwargs)

        return None  # no execution size == possize


In [ ]:
start_dt = dt.utcnow() - timedelta(minutes=4320)
end_dt = dt.utcnow()

execute(
    bases=["BTC"],
    quote="USDT",
    strategy=StreamML,
    start_dt=start_dt,
    end_dt=end_dt,
    interval='1m',
    debug=True,
    verbose=False,
    stratKwargs = dict(
        model=logisticReg,
        metric=metrics.ROCAUC,
        metric_window=100,
        log=True,
        debug=True

    )
)